<a href="https://colab.research.google.com/github/psygrammer/psypy_lm/blob/main/notebooks/ch01/ch01_start_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Getting Started with the Model Architecture of the Transformer

----------------

# Contents
* The background of the Transformer
* The rise of the Tranformer : Attention is All You Need
  - The encoder stack
    - Input embedding
    - Positional encoding
    - Sub-layer 1 : Multi-head attention
    - Sub-layer 2 : Feedforward network
  - The decoder stack
    - Output embedding and position encoding
    - The attention layers
    - The FFN sub-layer, the Post-LN, and the linear layer
* Training and performance
  - Before we end the chapter
* Summary

----------------

# The background of the Transformer

----------------

# The rise of the Tranformer : Attention is All You Need
* The encoder stack
* The decoder stack


--------------------

## The encoder stack
* Input embedding
* Positional encoding
* Sub-layer 1 : Multi-head attention
* Sub-layer 2 : Feedforward network


In [2]:
!git clone https://github.com/psygrammer/psypy_lm.git

Cloning into 'psypy_lm'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 27 (delta 6), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [3]:
!ls

psypy_lm  sample_data


In [4]:
# change work_dir
%cd /content/psypy_lm/notebooks/ch01/

/content/psypy_lm/notebooks/ch01


In [5]:
!ls

ch01_start_transformer.ipynb  text.txt


In [6]:
#!pip install --upgrade gensim
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
import math
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize 
import gensim 
from gensim.models import Word2Vec 
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings(action = 'ignore') 

### Input embedding


In [8]:
input_str = "the Transformer is an innovative NLP model!"

In [9]:
tokens = word_tokenize(input_str)
tokens

['the', 'Transformer', 'is', 'an', 'innovative', 'NLP', 'model', '!']

In [10]:
# word2vec을 사용하기 위해 학습을 해보자. 
# 아래처럼 코퍼스 파일이 있는 상황.
!ls

ch01_start_transformer.ipynb  text.txt


In [11]:
with open('text.txt', 'r') as sample:
  s = sample.read()

In [12]:
# processing escape characters 
f = s.replace("\n", " ")

In [13]:
data = []

# sentence parsing 
for i in sent_tokenize(f): 
	temp = [] 
	# tokenize the sentence into words 
	for j in word_tokenize(i): 
		temp.append(j.lower()) 
	data.append(temp) 

In [14]:
data[:2]

[['the',
  'black',
  'cat',
  'sat',
  'on',
  'the',
  'couch',
  'and',
  'the',
  'brown',
  'dog',
  'slept',
  'on',
  'the',
  'rug.the',
  'cat',
  'did',
  'not',
  'cross',
  'the',
  'street',
  'because',
  'it',
  'was',
  'too',
  'wet.the',
  'dog',
  'sat',
  'on',
  'the',
  'couch',
  'near',
  'the',
  'rug.the',
  'black',
  'cat',
  'sat',
  'on',
  'the',
  'couch',
  'and',
  'the',
  'brown',
  'dog',
  'slept',
  'on',
  'the',
  'rug.the',
  'cat',
  'did',
  'not',
  'cross',
  'the',
  'street',
  'because',
  'it',
  'was',
  'too',
  'wet.the',
  'dog',
  'sat',
  'on',
  'the',
  'couch',
  'near',
  'the',
  'rug',
  '.'],
 ['the',
  'black',
  'cat',
  'sat',
  'on',
  'the',
  'couch',
  'and',
  'the',
  'brown',
  'dog',
  'slept',
  'on',
  'the',
  'rug.the',
  'cat',
  'did',
  'not',
  'cross',
  'the',
  'street',
  'because',
  'it',
  'was',
  'too',
  'wet.the',
  'dog',
  'sat',
  'on',
  'the',
  'couch',
  'near',
  'the',
  'rug',
  '.']]

In [15]:
# Creating Skip Gram model 
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 512,window = 5, sg = 1) 

In [16]:
# 1-The 2-black 3-cat 4-sat 5-on 6-the 7-couch 8-and 9-the 10-brown 11-dog 12-slept 13-on 14-the 15-rug.
word1='black'
word2='brown'
pos1=2
pos2=10

In [17]:
a=model2[word1]
a

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-8.47721025e-02,  1.92767270e-02,  4.80953343e-02,  4.38695960e-02,
       -1.98584981e-02,  1.94863533e-04, -9.37078893e-03, -3.64531055e-02,
        4.29064631e-02,  2.25669444e-02, -2.85651349e-02,  4.39847447e-02,
       -4.92351875e-02,  1.11391984e-01, -6.20617568e-02, -2.16903780e-02,
        3.86966690e-02, -9.48898271e-02, -2.57886294e-02, -2.31535677e-02,
       -2.47979481e-02, -3.94748300e-02,  3.08977850e-02, -1.76938511e-02,
       -5.85205294e-02, -5.70257418e-02,  1.21840676e-02,  1.05485758e-02,
       -4.66045551e-02,  3.06124147e-02,  4.20735404e-02, -2.19136570e-02,
       -1.98060330e-02, -6.10877462e-02,  6.62884563e-02,  6.39225617e-02,
        4.21723500e-02, -1.27553465e-02,  3.97198659e-04, -5.12058511e-02,
        9.52383876e-02, -3.48878950e-02,  5.17646112e-02, -5.78341521e-02,
       -7.42598325e-02, -3.05307601e-02, -6.39610589e-02, -4.94734570e-02,
        2.02991217e-02, -1.06454730e-01, -2.68025231e-02, -3.09177544e-02,
       -4.47559170e-02, -

In [18]:
b=model2[word2]
b

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-8.56020153e-02,  1.85598321e-02,  4.97159921e-02,  4.30167951e-02,
       -1.96620058e-02, -9.56183067e-04, -9.09809768e-03, -3.70072909e-02,
        4.26636189e-02,  2.30442099e-02, -2.85518728e-02,  4.33949530e-02,
       -4.90924940e-02,  1.11303635e-01, -6.26749396e-02, -2.27291007e-02,
        3.85196656e-02, -9.37608406e-02, -2.49614976e-02, -2.33968087e-02,
       -2.53722202e-02, -3.83577496e-02,  3.10913455e-02, -1.73100736e-02,
       -5.72046414e-02, -5.57962060e-02,  1.29266074e-02,  1.14582125e-02,
       -4.80816327e-02,  3.05881985e-02,  4.34859507e-02, -2.23887712e-02,
       -1.80590544e-02, -6.22936785e-02,  6.69175759e-02,  6.42641112e-02,
        4.24699709e-02, -1.27701787e-02, -1.28085329e-03, -5.03569655e-02,
        9.60356519e-02, -3.35994251e-02,  5.13845049e-02, -5.63783087e-02,
       -7.41351694e-02, -3.14522721e-02, -6.50017932e-02, -5.08664064e-02,
        2.09558569e-02, -1.06218047e-01, -2.68299989e-02, -3.10174692e-02,
       -4.37910557e-02, -

In [21]:
# compute cosine similarity
dot = np.dot(a, b)
norma = np.linalg.norm(a)
normb = np.linalg.norm(b)
cos = dot / (norma * normb)
cos

0.9998744

In [22]:
a.shape

(512,)

In [24]:
a

array([-8.47721025e-02,  1.92767270e-02,  4.80953343e-02,  4.38695960e-02,
       -1.98584981e-02,  1.94863533e-04, -9.37078893e-03, -3.64531055e-02,
        4.29064631e-02,  2.25669444e-02, -2.85651349e-02,  4.39847447e-02,
       -4.92351875e-02,  1.11391984e-01, -6.20617568e-02, -2.16903780e-02,
        3.86966690e-02, -9.48898271e-02, -2.57886294e-02, -2.31535677e-02,
       -2.47979481e-02, -3.94748300e-02,  3.08977850e-02, -1.76938511e-02,
       -5.85205294e-02, -5.70257418e-02,  1.21840676e-02,  1.05485758e-02,
       -4.66045551e-02,  3.06124147e-02,  4.20735404e-02, -2.19136570e-02,
       -1.98060330e-02, -6.10877462e-02,  6.62884563e-02,  6.39225617e-02,
        4.21723500e-02, -1.27553465e-02,  3.97198659e-04, -5.12058511e-02,
        9.52383876e-02, -3.48878950e-02,  5.17646112e-02, -5.78341521e-02,
       -7.42598325e-02, -3.05307601e-02, -6.39610589e-02, -4.94734570e-02,
        2.02991217e-02, -1.06454730e-01, -2.68025231e-02, -3.09177544e-02,
       -4.47559170e-02, -

In [23]:
aa = a.reshape(1,512)
aa 

array([[-8.47721025e-02,  1.92767270e-02,  4.80953343e-02,
         4.38695960e-02, -1.98584981e-02,  1.94863533e-04,
        -9.37078893e-03, -3.64531055e-02,  4.29064631e-02,
         2.25669444e-02, -2.85651349e-02,  4.39847447e-02,
        -4.92351875e-02,  1.11391984e-01, -6.20617568e-02,
        -2.16903780e-02,  3.86966690e-02, -9.48898271e-02,
        -2.57886294e-02, -2.31535677e-02, -2.47979481e-02,
        -3.94748300e-02,  3.08977850e-02, -1.76938511e-02,
        -5.85205294e-02, -5.70257418e-02,  1.21840676e-02,
         1.05485758e-02, -4.66045551e-02,  3.06124147e-02,
         4.20735404e-02, -2.19136570e-02, -1.98060330e-02,
        -6.10877462e-02,  6.62884563e-02,  6.39225617e-02,
         4.21723500e-02, -1.27553465e-02,  3.97198659e-04,
        -5.12058511e-02,  9.52383876e-02, -3.48878950e-02,
         5.17646112e-02, -5.78341521e-02, -7.42598325e-02,
        -3.05307601e-02, -6.39610589e-02, -4.94734570e-02,
         2.02991217e-02, -1.06454730e-01, -2.68025231e-0

In [25]:
ba = b.reshape(1,512)

In [27]:
cos_lib = cosine_similarity(aa, ba)
cos_lib

array([[0.9998744]], dtype=float32)

### Positional encoding


In [28]:
pe1=aa.copy()
pe2=aa.copy()
pe3=aa.copy()
paa=aa.copy()
pba=ba.copy()
d_model=512
max_print=d_model
max_length=20

In [33]:
pe1[0]

array([-8.47721025e-02,  1.92767270e-02,  4.80953343e-02,  4.38695960e-02,
       -1.98584981e-02,  1.94863533e-04, -9.37078893e-03, -3.64531055e-02,
        4.29064631e-02,  2.25669444e-02, -2.85651349e-02,  4.39847447e-02,
       -4.92351875e-02,  1.11391984e-01, -6.20617568e-02, -2.16903780e-02,
        3.86966690e-02, -9.48898271e-02, -2.57886294e-02, -2.31535677e-02,
       -2.47979481e-02, -3.94748300e-02,  3.08977850e-02, -1.76938511e-02,
       -5.85205294e-02, -5.70257418e-02,  1.21840676e-02,  1.05485758e-02,
       -4.66045551e-02,  3.06124147e-02,  4.20735404e-02, -2.19136570e-02,
       -1.98060330e-02, -6.10877462e-02,  6.62884563e-02,  6.39225617e-02,
        4.21723500e-02, -1.27553465e-02,  3.97198659e-04, -5.12058511e-02,
        9.52383876e-02, -3.48878950e-02,  5.17646112e-02, -5.78341521e-02,
       -7.42598325e-02, -3.05307601e-02, -6.39610589e-02, -4.94734570e-02,
        2.02991217e-02, -1.06454730e-01, -2.68025231e-02, -3.09177544e-02,
       -4.47559170e-02, -

In [36]:
for i in range(0, max_print,2):
  pe1[0][i] = math.sin(pos1 / (10000 ** ((2 * i)/d_model)))
  paa[0][i] = (paa[0][i]*math.sqrt(d_model))+ pe1[0][i]
  pe1[0][i+1] = math.cos(pos1 / (10000 ** ((2 * i)/d_model)))
  paa[0][i+1] = (paa[0][i+1]*math.sqrt(d_model))+pe1[0][i+1]
  print(i,pe1[0][i],i+1,pe1[0][i+1])
  print(i,paa[0][i],i+1,paa[0][i+1])
  print("\n")

0 0.9092974 1 -0.41614684
0 -1.0088763 1 0.020035695


2 0.95814437 3 -0.28628543
2 2.0464175 3 0.70637023


4 0.98704624 5 -0.16043596
4 0.5376997 5 -0.1560267


6 0.9991642 7 -0.040876657
6 0.7871275 7 -0.8657163


8 0.99748 9 0.07094825
8 1.9683424 9 0.5815799


10 0.984703 11 0.17424123
10 0.3383478 11 1.1695024


12 0.9632266 13 0.2686903
12 -0.1508385 13 2.7892032


14 0.9351183 15 0.35433567
14 -0.46917894 15 -0.13646156


16 0.9021307 17 0.43146282
16 1.7777364 17 -1.7156489


18 0.8657256 19 0.5005189
18 0.2821955 19 -0.023386514


20 0.8271038 21 0.5620492
20 0.2659903 21 -0.33116424


22 0.7872378 23 0.6166495
22 1.4863749 23 0.21628337


24 0.74690354 25 0.6649324
24 -0.5772649 25 -0.6254128


26 0.7067105 27 0.7075028
26 0.9824045 27 0.9461898


28 0.6671291 29 0.7449421
28 -0.3874116 29 1.437622


30 0.62851435 31 0.777798
30 1.5805299 31 0.28194854


32 0.5911271 33 0.8065784
32 0.14296773 33 -0.5756795


34 0.55515176 35 0.8317491
34 2.0550883 35 2.2781515


36 0.520711

In [38]:
#print(pe1)
# A  method in Pytorch using torch.exp and math.log :
max_len=max_length                
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
#print(pe[:, 0::2])

In [39]:
for i in range(0, max_print,2):
  pe2[0][i] = math.sin(pos2 / (10000 ** ((2 * i)/d_model)))
  pba[0][i] = (pba[0][i]*math.sqrt(d_model))+ pe2[0][i]
            
  pe2[0][i+1] = math.cos(pos2 / (10000 ** ((2 * i)/d_model)))
  pba[0][i+1] = (pba[0][i+1]*math.sqrt(d_model))+ pe2[0][i+1]
               
  #print(i,pe2[0][i],i+1,pe2[0][i+1])
  #print(i,paa[0][i],i+1,paa[0][i+1])
  #print("\n")

In [40]:
print(word1,word2)
cos_lib = cosine_similarity(aa, ba)
print(cos_lib,"word similarity")
cos_lib = cosine_similarity(pe1, pe2)
print(cos_lib,"positional similarity")
cos_lib = cosine_similarity(paa, pba)
print(cos_lib,"positional encoding similarity")

black brown
[[0.9998744]] word similarity
[[0.8600013]] positional similarity
[[0.9633899]] positional encoding similarity


In [41]:
print(word1)
print("embedding")
print(aa)
print("positional encoding")
print(pe1)
print("encoded embedding")
print(paa)

print("========================")

print(word2)
print("embedding")
print(ba)
print("positional encoding")
print(pe2)
print("encoded embedding")
print(pba)

black
embedding
[[-8.47721025e-02  1.92767270e-02  4.80953343e-02  4.38695960e-02
  -1.98584981e-02  1.94863533e-04 -9.37078893e-03 -3.64531055e-02
   4.29064631e-02  2.25669444e-02 -2.85651349e-02  4.39847447e-02
  -4.92351875e-02  1.11391984e-01 -6.20617568e-02 -2.16903780e-02
   3.86966690e-02 -9.48898271e-02 -2.57886294e-02 -2.31535677e-02
  -2.47979481e-02 -3.94748300e-02  3.08977850e-02 -1.76938511e-02
  -5.85205294e-02 -5.70257418e-02  1.21840676e-02  1.05485758e-02
  -4.66045551e-02  3.06124147e-02  4.20735404e-02 -2.19136570e-02
  -1.98060330e-02 -6.10877462e-02  6.62884563e-02  6.39225617e-02
   4.21723500e-02 -1.27553465e-02  3.97198659e-04 -5.12058511e-02
   9.52383876e-02 -3.48878950e-02  5.17646112e-02 -5.78341521e-02
  -7.42598325e-02 -3.05307601e-02 -6.39610589e-02 -4.94734570e-02
   2.02991217e-02 -1.06454730e-01 -2.68025231e-02 -3.09177544e-02
  -4.47559170e-02 -5.96076772e-02 -4.37319018e-02  5.80162928e-02
  -6.04955829e-04 -6.73722848e-02  1.60909537e-02  3.1715139

### Sub-layer 1 : Multi-head attention


##### Step 1: Represent the input

In [2]:
import numpy as np
from scipy.special import softmax

In [3]:
print("Step 1: Input : 3 inputs, d_model=4")
x =np.array([[1.0, 0.0, 1.0, 0.0],   # Input 1
             [0.0, 2.0, 0.0, 2.0],   # Input 2
             [1.0, 1.0, 1.0, 1.0]])  # Input 3
print(x)

Step 1: Input : 3 inputs, d_model=4
[[1. 0. 1. 0.]
 [0. 2. 0. 2.]
 [1. 1. 1. 1.]]


##### Step 2: Initializing the weight matrices

In [4]:
print("Step 2: weights 3 dimensions x d_model=4")
print("w_query")
w_query =np.array([[1, 0, 1],
                   [1, 0, 0],
                   [0, 0, 1],
                   [0, 1, 1]])
print(w_query)

Step 2: weights 3 dimensions x d_model=4
w_query
[[1 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 1]]


In [5]:
print("w_key")
w_key =np.array([[0, 0, 1],
                 [1, 1, 0],
                 [0, 1, 0],
                 [1, 1, 0]])
print(w_key)

w_key
[[0 0 1]
 [1 1 0]
 [0 1 0]
 [1 1 0]]


In [6]:
print("w_value")
w_value = np.array([[0, 2, 0],
                    [0, 3, 0],
                    [1, 0, 3],
                    [1, 1, 0]])
print(w_value)

w_value
[[0 2 0]
 [0 3 0]
 [1 0 3]
 [1 1 0]]


##### Step 3: Matrix multiplication to obtain Q, K, V

In [7]:
print("Step 3: Matrix multiplication to obtain Q,K,V")
print("Query: x * w_query")
Q=np.matmul(x,w_query)
print(Q)

Step 3: Matrix multiplication to obtain Q,K,V
Query: x * w_query
[[1. 0. 2.]
 [2. 2. 2.]
 [2. 1. 3.]]


In [8]:
print("Key: x * w_key")
K=np.matmul(x,w_key)
print(K)

Key: x * w_key
[[0. 1. 1.]
 [4. 4. 0.]
 [2. 3. 1.]]


In [9]:
print("Value: x * w_value")
V=np.matmul(x,w_value)
print(V)

Value: x * w_value
[[1. 2. 3.]
 [2. 8. 0.]
 [2. 6. 3.]]


##### Step 4: Scaled attention scores

In [10]:
print("Step 4: Scaled Attention Scores")
k_d = 1 #square root of k_d=3 rounded down to 1 for this example 
attention_scores = (Q @ K.transpose())/k_d 
print(attention_scores)

Step 4: Scaled Attention Scores
[[ 2.  4.  4.]
 [ 4. 16. 12.]
 [ 4. 12. 10.]]


##### Step 5: Scaled softmax attention scores for each vector

In [11]:
print("Step 5: Scaled softmax attention_scores for each vector")
attention_scores[0]=softmax(attention_scores[0])
attention_scores[1]=softmax(attention_scores[1])
attention_scores[2]=softmax(attention_scores[2])
print(attention_scores[0])
print(attention_scores[1])
print(attention_scores[2])

Step 5: Scaled softmax attention_scores for each vector
[0.06337894 0.46831053 0.46831053]
[6.03366485e-06 9.82007865e-01 1.79861014e-02]
[2.95387223e-04 8.80536902e-01 1.19167711e-01]


##### Step 6: The final attention representations

In [13]:
print("Step 6: attention value obtained by score1/k_d * V")
print(V[0])
print(V[1])
print(V[2])

Step 6: attention value obtained by score1/k_d * V
[1. 2. 3.]
[2. 8. 0.]
[2. 6. 3.]


In [14]:
print("Attention 1")
attention1=attention_scores[0].reshape(-1,1)
attention1=attention_scores[0][0]*V[0]
print(attention1)

Attention 1
[0.06337894 0.12675788 0.19013681]


In [15]:
print("Attention 2")
attention2=attention_scores[0][1]*V[1]
print(attention2)

Attention 2
[0.93662106 3.74648425 0.        ]


In [16]:
print("Attention 3")
attention3=attention_scores[0][2]*V[2]
print(attention3)

Attention 3
[0.93662106 2.80986319 1.40493159]


##### Step 7: Summing up the results

In [17]:
print("Step7: summed the results to create the first line of the output matrix")
attention_input1=attention1+attention2+attention3
print(attention_input1)

Step7: summed the results to create the first line of the output matrix
[1.93662106 6.68310531 1.59506841]


##### Step 8: Steps 1 to 7 for all the inputs

In [19]:
print("Step 8: Step 1 to 7 for inputs 1 to 3")
#We assume we have 3 results with learned weights (they were not trained in this example)
#We assume we are implementing the original Transformer paper.We will have 3 results of 64 dimensions each 
attention_head1=np.random.random((3, 64))
print(attention_head1)

Step 8: Step 1 to 7 for inputs 1 to 3
[[0.98550421 0.69239609 0.41630611 0.77804287 0.24200857 0.67972555
  0.53339776 0.89381814 0.68144074 0.32319275 0.05732555 0.53927535
  0.00385359 0.78189817 0.29055389 0.0443541  0.85531549 0.91210496
  0.83753919 0.80081719 0.33810596 0.83194161 0.49140682 0.42388299
  0.72766921 0.77616781 0.47770755 0.28895931 0.36764413 0.64254159
  0.33070623 0.39721833 0.79803024 0.72290943 0.1831662  0.30067515
  0.46350314 0.28110154 0.49200543 0.28043443 0.63799939 0.97766918
  0.79009447 0.80628712 0.21188533 0.04959939 0.7192935  0.72066895
  0.16076112 0.40831617 0.07500193 0.88284874 0.42545103 0.84235532
  0.72652677 0.63126199 0.29851319 0.56029458 0.30546748 0.14275568
  0.4424857  0.67414772 0.8411103  0.73755977]
 [0.06845164 0.7830146  0.30075872 0.92254343 0.84102332 0.53608184
  0.48808664 0.31126187 0.41049226 0.88877125 0.55029036 0.61472428
  0.44580105 0.5125898  0.87953088 0.79155344 0.81651916 0.22353947
  0.34829946 0.40479455 0.33525

##### Step 9: The output of the heads of the attention sub-layer

In [21]:
print("Step 9: We assume we have trained the 8 heads of the attention sub-layer")
z0h1=np.random.random((3, 64))
z1h2=np.random.random((3, 64))
z2h3=np.random.random((3, 64))
z3h4=np.random.random((3, 64))
z4h5=np.random.random((3, 64))
z5h6=np.random.random((3, 64))
z6h7=np.random.random((3, 64))
z7h8=np.random.random((3, 64))
print("shape of one head",z0h1.shape,"dimension of 8 heads",64*8)

Step 9: We assume we have trained the 8 heads of the attention sub-layer
shape of one head (3, 64) dimension of 8 heads 512


##### Step 10: Concatenation of the output of the heads

In [22]:
print("Step 10: Concantenation of heads 1 to 8 to obtain the original 8x64=512 ouput dimension of the model")
output_attention=np.hstack((z0h1,z1h2,z2h3,z3h4,z4h5,z5h6,z6h7,z7h8))
print(output_attention)

Step 10: Concantenation of heads 1 to 8 to obtain the original 8x64=512 ouput dimension of the model
[[0.41873839 0.18751283 0.40838379 ... 0.16434145 0.60697705 0.52614693]
 [0.59202435 0.44543335 0.32357525 ... 0.80058391 0.58525933 0.78519285]
 [0.48441961 0.3221019  0.49370963 ... 0.00242846 0.94489259 0.22310239]]


### Sub-layer 2 : Feedforward network


## The decoder stack
* Output embedding and position encoding
* The attention layers
* The FFN sub-layer, the Post-LN, and the linear layer


### Output embedding and position encoding


### The attention layers


### The FFN sub-layer, the Post-LN, and the linear layer


# Training and performance
* Before we end the chapter


# Summary